## Deliverable 2. Create a Customer Travel Destinations Map.

In [118]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [120]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,palestine,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Zaysan,KZ,47.4667,84.8667,0.73,89,85,0.63,overcast clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,81.64,76,42,12.86,light rain
2,2,Paita,PE,-5.0892,-81.1144,75.54,74,69,12.82,broken clouds
3,3,Leningradskiy,RU,69.3833,178.4167,-23.17,85,100,8.05,overcast clouds
4,4,Hermanus,ZA,-34.4187,19.2345,64.99,81,72,3.00,broken clouds
5,5,Mataura,NZ,-46.1927,168.8643,70.48,36,0,0.94,clear sky
6,6,Ushuaia,AR,-54.8000,-68.3000,42.80,75,75,17.27,broken clouds
7,7,Begoro,GH,6.3871,-0.3774,70.84,92,30,2.66,scattered clouds
8,8,Touros,BR,-5.1989,-35.4608,78.80,78,20,3.44,few clouds
9,9,Bluff,NZ,-46.6000,168.3333,54.30,72,0,14.85,clear sky


In [121]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature of the city you would like to travel to?"))
max_temp=float(input("What is the maximum temperature of the city you would like the travel to?"))

What is the minimum temperature of the city you would like to travel to?55
What is the maximum temperature of the city you would like the travel to?85


In [122]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temperature"] <= max_temp) & \
                                       (city_data_df["Max Temperature"] >= min_temp)].dropna()
preferred_cities_df

,palestine,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hithadhoo,MV,-0.6000,73.0833,81.64,76,42,12.86,light rain
2,2,Paita,PE,-5.0892,-81.1144,75.54,74,69,12.82,broken clouds
4,4,Hermanus,ZA,-34.4187,19.2345,64.99,81,72,3.00,broken clouds
5,5,Mataura,NZ,-46.1927,168.8643,70.48,36,0,0.94,clear sky
7,7,Begoro,GH,6.3871,-0.3774,70.84,92,30,2.66,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
695,695,Palora,EC,-1.7000,-77.9333,66.78,94,32,2.46,moderate rain
697,697,Usoke,TZ,-5.1667,32.3500,63.43,97,93,4.34,overcast clouds
698,698,Puerto Escondido,MX,15.8500,-97.0667,80.60,83,90,9.22,overcast clouds
699,699,Angoche,MZ,-16.2325,39.9086,78.01,86,98,2.62,light rain


In [123]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

palestine              0
City                   0
Country                0
Latitude               0
Longitude              0
Max Temperature        0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [124]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temperature", "Current Description", "Latitude", "Longitude"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temperature,Current Description,Latitude,Longitude,Hotel Name
1,Hithadhoo,MV,81.64,light rain,-0.6000,73.0833,
2,Paita,PE,75.54,broken clouds,-5.0892,-81.1144,
4,Hermanus,ZA,64.99,broken clouds,-34.4187,19.2345,
5,Mataura,NZ,70.48,clear sky,-46.1927,168.8643,
7,Begoro,GH,70.84,scattered clouds,6.3871,-0.3774,
8,Touros,BR,78.80,few clouds,-5.1989,-35.4608,
12,Airai,TL,63.90,overcast clouds,-8.9266,125.4092,
14,Hilo,US,73.40,moderate rain,19.7297,-155.0900,
16,Ribeira Grande,PT,61.07,overcast clouds,38.5167,-28.7000,
18,Avarua,CK,84.20,light rain,-21.2078,-159.7750,


In [125]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame 
    lat=row["Latitude"]
    lng=row["Longitude"]
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url,params=params).json()
    hotels
#     # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found.. skipping hotel..")
        

Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. skipping hotel..
Hotel not found.. sk

In [126]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df[hotel_df["Hotel Name"]!=""]
clean_hotel_df.head(10)

,City,Country,Max Temperature,Current Description,Latitude,Longitude,Hotel Name
1,Hithadhoo,MV,81.64,light rain,-0.6000,73.0833,Scoop Guest House
2,Paita,PE,75.54,broken clouds,-5.0892,-81.1144,Hotel Náutico de Paita
4,Hermanus,ZA,64.99,broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
5,Mataura,NZ,70.48,clear sky,-46.1927,168.8643,Ellie's Villa
7,Begoro,GH,70.84,scattered clouds,6.3871,-0.3774,MR ODAME
8,Touros,BR,78.80,few clouds,-5.1989,-35.4608,INN NEW HORIZON
12,Airai,TL,63.90,overcast clouds,-8.9266,125.4092,Juvinal Martin
14,Hilo,US,73.40,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
16,Ribeira Grande,PT,61.07,overcast clouds,38.5167,-28.7000,Quinta da Meia Eira
18,Avarua,CK,84.20,light rain,-21.2078,-159.7750,Paradise Inn


In [127]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [128]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temperature</dt><dd>{Max Temperature}</dd>
<dt>Weather Description</dt><dd>{Current Description}F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Latitude", "Longitude"]]

In [129]:
# 11a. Add a marker layer for each city to the map. 
max_temp=clean_hotel_df["Max Temperature"]
fig=gmaps.figure(center=(30.00,31.00),zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations,weights=max_temp,dissipating=False,point_radius=1.5,max_intensity=300)
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# # 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))